# Load dependencies:

In [1]:
import cobra
import libsbml
import lxml

models_directory = '/media/sf_Shared/Systems_biology/Metabolic_models/'
OB3b_directory = '/media/sf_Shared/GEM_OB3b/'
memote_directory = '/home/ensakz/Desktop/memote_m_trichosporium/'
fastas_directory = '/media/sf_Shared/Systems_biology/Fastas_and_annotations/'
inparanoid_directory = '/media/sf_Shared/Systems_biology/InParanoid_runs/'
draft_gems_dir_8 = 'draft_gems/8.OB3b_draft_gem_during_pathway_manual_curation/'
gem_confirmation = 'gem_confirmation/'

In [66]:
iMsOB3b = cobra.io.load_json_model(memote_directory + draft_gems_dir_8 + "bigg_OB3b_12_02_2020.json")

In [3]:
model_20Z = cobra.io.read_sbml_model(models_directory + "i20ZR.xml")

In [189]:
model_universal = cobra.io.load_json_model(models_directory + 'universal_model.json')

# Make acetyl-CoA come from 2 separate sources:
MALCOAL and pyruvate dehydrogenase

Check how similar strategy is implemented in 20Z model:

In [135]:
print(model_20Z.metabolites.pyr_c.formula)
print(model_20Z.metabolites.pyra_c.formula)
print(model_20Z.metabolites.pyrb_c.formula)

C3H3O3
C3H4O3
C3H4O3


Implement similar strategy to OB3b model:

In [136]:
iMsOB3b.metabolites.accoa_c.summary()

PRODUCING REACTIONS -- acetyl_CoA (accoa_c)
-------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
99%  6.27    MALCOAL   malScoa_c <=> accoa_c + glx_c
1%   0.0653  PTAr      accoa_c + pi_c <=> actp_c + coa_c

CONSUMING REACTIONS -- acetyl_CoA (accoa_c)
-------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
73%  4.63    CS        accoa_c + h2o_c + oaa_c <=> cit_c + coa_c + h_c
24%  1.51    ACACT1r   2.0 accoa_c <=> aacoa_c + coa_c
2%   0.119   ACCOACr   accoa_c + atp_c + hco3_c --> adp_c + h_c + malc...


In [137]:
iMsOB3b.summary()

IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  26.4   Biomass_Mext...  0.123
ch4_e      14.9       co2_e   9.97
h_e         7.3
no3_e       1.2
pi_e        0.0857
k_e         0.0226
so4_e       0.00774
mg2_e       0.00101
fe2_e       0.000984
ca2_e       0.000607
cl_e        0.000607
na1_e       0.000499
mobd_e      0.000432
cu2_e       0.000405
mn2_e       0.000405
zn2_e       0.000405
cobalt2_e   2.69e-05


In [140]:
iMsOB3b.reactions.MALCOAL.reaction = 'malScoa_c <=> accoaa_c + glx_c'

In [141]:
iMsOB3b.reactions.PDH.reaction = 'coa_c + nad_c + pyr_c --> accoab_c + co2_c + nadh_c'

In [142]:
iMsOB3b.metabolites.coa_c.summary()

PRODUCING REACTIONS -- CoA (coa_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
43%  1.25    PHAS      r3hbcoa_c --> coa_c + pbhb_c
26%  0.757   ACACT1r   2.0 accoa_c <=> aacoa_c + coa_c
23%  0.662   CS        accoa_c + h2o_c + oaa_c <=> cit_c + coa_c + h_c
4%   0.119   MCOATA    ACP_c + malcoa_c <=> coa_c + malACP_c
2%   0.0487  IPPS2     3mob_c + accoa_c + h2o_c --> 2ippa_c + coa_c + h_c
1%   0.0316  THDPS     h2o_c + succoa_c + thdp_c <=> coa_c + sl2a6o_c

CONSUMING REACTIONS -- CoA (coa_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
83%  2.37    PTAr      accoa_c + pi_c <=> actp_c + coa_c
18%  0.521   AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...


In [166]:
iMsOB3b.metabolites.accoa_c.summary()

PRODUCING REACTIONS -- acetyl_CoA (accoa_c)
-------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
98%  2.68    ACSPW     2.0 accoaa_c + accoab_c --> 3.0 accoa_c
2%   0.0653  PTAr      accoa_c + pi_c <=> actp_c + coa_c

CONSUMING REACTIONS -- acetyl_CoA (accoa_c)
-------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
88%  2.41    ACACT1r   2.0 accoa_c <=> aacoa_c + coa_c
5%   0.141   CS        accoa_c + h2o_c + oaa_c <=> cit_c + coa_c + h_c
4%   0.108   ACCOACr   accoa_c + atp_c + hco3_c --> adp_c + h_c + malc...
2%   0.0487  IPPS2     3mob_c + accoa_c + h2o_c --> 2ippa_c + coa_c + h_c


In [167]:
iMsOB3b.reactions.ACACT1r.flux

1.205137816797321

In [145]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('ACSPW')
reaction.name = 'Acetyl-CoA synthesis from 2 pathways'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [146]:
iMsOB3b.add_reaction(reaction)
iMsOB3b.reactions.ACSPW.reaction = '2 accoaa_c + accoab_c --> 3 accoa_c'

In [154]:
iMsOB3b.reactions.ACSPW.flux

0.8927475649452175

In [176]:
iMsOB3b.metabolites.pyr_c.summary()

PRODUCING REACTIONS -- pyruvate (pyr_c)
---------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%  1.29    PYK       atp_c + pyr_c <=> adp_c + h_c + pep_c

CONSUMING REACTIONS -- pyruvate (pyr_c)
---------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
69%   0.893   PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
16%   0.2     ACLS      h_c + 2.0 pyr_c <=> alac_S_c + co2_c
11%   0.135   ALATA_D   glu_D_c + pyr_c <=> akg_c + ala_D_c
2%    0.0316  DHDPS     aspsa_c + pyr_c <=> 23dhdp_c + 2.0 h2o_c + h_c
2%    0.0256  ACHBS     2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c


In [170]:
with iMsOB3b as model:
    #model.reactions.PTAr.bounds = (0, 0)
    #print(model.reactions.ACSPW)
    #model.reactions.ACS.bounds = (0, 0)
    #model.reactions.ACSPW.bounds = (0, 0)
    #solution = 
    model.reactions.FALDA.bounds = (0, 0)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(memote_directory + 'gem_confirmation/additional_constraints/' +
                    'acetyl_coA_flux_33_from_pyruvate.csv')
    model.metabolites.accoa_c.summary()
    print(solution['ACACT1r'])

PRODUCING REACTIONS -- acetyl_CoA (accoa_c)
-------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
98%  2.68    ACSPW     2.0 accoaa_c + accoab_c --> 3.0 accoa_c
2%   0.0653  PTAr      accoa_c + pi_c <=> actp_c + coa_c

CONSUMING REACTIONS -- acetyl_CoA (accoa_c)
-------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
88%  2.41    ACACT1r   2.0 accoa_c <=> aacoa_c + coa_c
5%   0.141   CS        accoa_c + h2o_c + oaa_c <=> cit_c + coa_c + h_c
4%   0.108   ACCOACr   accoa_c + atp_c + hco3_c --> adp_c + h_c + malc...
2%   0.0487  IPPS2     3mob_c + accoa_c + h2o_c --> 2ippa_c + coa_c + h_c
1.14979236729033


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:10 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


Save iMsOB3b model:

In [171]:
cobra.io.save_json_model(iMsOB3b, memote_directory + draft_gems_dir_8 + "bigg_OB3b_12_02_2020.json")

# Partially reversible TCA:
Specific growth rate increased significantly

In [190]:
model_universal.reactions.OOR3r.bounds = (-1000, 1000)
model_universal.reactions.OOR3r

Reaction identifier,OOR3r
Name,2-oxoglutarate synthase (rev)
Memory address,0x07f1f3af2e1d0
Stoichiometry,akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd_c + succoa_c 2-Oxoglutarate + Coenzyme A + Oxidized ferredoxin + H+ <=> CO2 CO2 + Reduced ferredoxin + Succinyl-CoA
GPR,
Lower bound,-1000
Upper bound,1000


In [192]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 0)
    model.reactions.SUCOAS.bounds = (-1000, 1000)
    model.add_reaction(model_universal.reactions.OOR3r)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(memote_directory + 'gem_confirmation/additional_constraints/' +
                    'partially_reverse_tca.csv')
    print(solution['Biomass_Mextorquens_AM1_core'])

0.15474544176219604


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


Save iMsOB3b, partially reversible TCA:

In [198]:
model.add_reaction(model_universal.reactions.OOR3r)

In [199]:
cobra.io.save_json_model(iMsOB3b, memote_directory + draft_gems_dir_8 + "bigg_OB3b_12_02_2020_TCA_rev.json")

# Check CO2 supplementation:

## With 1/3 of acetyl-CoA coming from pyruvate:

In [146]:
iMsOB3b = cobra.io.load_json_model(memote_directory + draft_gems_dir_8 + "bigg_OB3b_12_02_2020.json")

In [47]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_co2_c')
reaction.name = 'Demand carbon dioxide'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [48]:
for r in np.arange(0, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 0)
        model.add_reaction(reaction)
        model.reactions.get_by_id('DM_co2_c').reaction = '--> co2_c'
        model.reactions.DM_co2_c.bounds = (r, 1000)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(model.reactions.Biomass_Mextorquens_AM1_core.flux)
        model.metabolites.co2_c.summary()

0.12260378253188546
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
92%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
7%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.141  ICDHyr    icit_c + nadp_c --> akg_c + co2_c + nadph_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
76%   9.97   CO2tpp    co2_p <=> co2_c
10%   1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
9%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
7%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188603
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
86%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
7%    1      DM_co2_c  --> co2_c
6%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.141  ICDHyr    icit_c + nadp_c --> akg_c + co2_c + nadph_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
78%  11      CO2tpp    co2_p <=> co2_c
9%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
8%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
7%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  -------------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
80%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
13%   2      DM_co2_c  --> co2_c
6%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.141  ICDHyr    icit_c + nadp_c --> akg_c + co2_c + nadph_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
79%  12      CO2tpp    co2_p <=> co2_c
9%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
8%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
6%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  -------------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
75%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
19%   3      DM_co2_c  --> co2_c
6%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.141  ICDHyr    icit_c + nadp_c --> akg_c + co2_c + nadph_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
81%  13      CO2tpp    co2_p <=> co2_c
8%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
7%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
6%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  -------------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
71%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
23%   4      DM_co2_c  --> co2_c
5%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.141  ICDHyr    icit_c + nadp_c --> akg_c + co2_c + nadph_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
82%  14      CO2tpp    co2_p <=> co2_c
8%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
7%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
6%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  -------------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
67%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
28%   5.1    DM_co2_c  --> co2_c
5%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
84%  15.1    CO2tpp    co2_p <=> co2_c
7%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
66%  12      FDHr      for_c + nad_c <=> co2_c + nadh

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
63%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
32%   6.2    DM_co2_c  --> co2_c
5%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
85%  16.2    CO2tpp    co2_p <=> co2_c
7%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
63%  12      FDHr      for_c + nad_c <=> co2_c + nadh

0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
59%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
36%   7.4    DM_co2_c  --> co2_c
4%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
85%  17.4    CO2tpp    co2_p <=> co2_c
7%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
59%  12      FDHr      for_c + na

0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
56%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
40%   8.5    DM_co2_c  --> co2_c
4%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
86%  18.5    CO2tpp    co2_p <=> co2_c
6%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
56%  12      FDHr      for_c + na

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
53%  12      FDHr      for_c + nad_c <=> co2_c + nadh_c
43%   9.6    DM_co2_c  --> co2_c
4%    0.893  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
87%  19.6    CO2tpp    co2_p <=> co2_c
6%    1.33   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%    1.17   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%    0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12260378253188493
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
53%  12      FDHr      for_c + nad_c <=> co2_c + nadh

## With reverse TCA:

### With OOR3r added:

In [49]:
iMsOB3b = cobra.io.load_json_model(memote_directory + draft_gems_dir_8 + "bigg_OB3b_12_02_2020_TCA_rev.json")

In [50]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_co2_c')
reaction.name = 'Demand carbon dioxide'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [51]:
for r in np.arange(0, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 0)
        model.add_reaction(reaction)
        model.reactions.get_by_id('DM_co2_c').reaction = '--> co2_c'
        model.reactions.DM_co2_c.bounds = (r, 1000)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(model.reactions.Biomass_Mextorquens_AM1_core.flux)
        model.metabolites.co2_c.summary()

0.15474544176219832
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
57%  11.5    FDHr      for_c + nad_c <=> co2_c + nadh_c
38%   7.63   AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
5%    1.04   PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.126  ACLS      h_c + 2.0 pyr_c <=> alac_S_c + co2_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
43%   8.68   CO2tpp    co2_p <=> co2_c
38%   7.81   OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
8%    1.63   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
7%    1.43   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    1.06   PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219862
PRODUCING REACTIONS -- CO2 (co2_c)

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
54%  11.5    FDHr      for_c + nad_c <=> co2_c + nadh_c
36%   7.63   AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
5%    1.04   PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
4%    0.9    DM_co2_c  --> co2_c
1%    0.126  ACLS      h_c + 2.0 pyr_c <=> alac_S_c + co2_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
45%   9.58   CO2tpp    co2_p <=> co2_c
37%   7.81   OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
8%    1.63   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
7%    1.43   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    1.06   PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219565
PRODUCING REACTIONS -

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
52%  11.5    FDHr      for_c + nad_c <=> co2_c + nadh_c
35%   7.63   AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
8%    1.8    DM_co2_c  --> co2_c
5%    1.04   PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.126  ACLS      h_c + 2.0 pyr_c <=> alac_S_c + co2_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
47%  10.5    CO2tpp    co2_p <=> co2_c
35%   7.81   OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
7%    1.63   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.43   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    1.06   PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219565
PRODUCING REACTIONS -

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
50%  11.5    FDHr      for_c + nad_c <=> co2_c + nadh_c
33%   7.63   AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
12%   2.7    DM_co2_c  --> co2_c
5%    1.04   PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.126  ACLS      h_c + 2.0 pyr_c <=> alac_S_c + co2_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
49%  11.4    CO2tpp    co2_p <=> co2_c
34%   7.81   OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
7%    1.63   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.43   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    1.06   PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219565
PRODUCING REACTIONS -

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
48%  11.5    FDHr      for_c + nad_c <=> co2_c + nadh_c
32%   7.63   AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
15%   3.6    DM_co2_c  --> co2_c
4%    1.04   PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.126  ACLS      h_c + 2.0 pyr_c <=> alac_S_c + co2_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
51%  12.3    CO2tpp    co2_p <=> co2_c
33%   7.81   OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
7%    1.63   HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.43   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%    1.06   PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219565
PRODUCING REACTIONS -

...

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
43%   11.5   FDHr      for_c + nad_c <=> co2_c + nadh_c
29%    7.63  AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
24%    6.3   DM_co2_c  --> co2_c
4%     1.04  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
57%   15     CO2tpp    co2_p <=> co2_c
29%    7.81  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
6%     1.63  HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%     1.43  CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%     1.06  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219565
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
42%   11.5   FDHr      for_c + nad_c <=> co2_c + nadh_c
28%    7.63  AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
26%    7.2   DM_co2_c  --> co2_c
4%     1.04  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
58%   15.9   CO2tpp    co2_p <=> co2_c
29%    7.81  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
6%     1.63  HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%     1.43  CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%     1.06  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219565
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
41%   11.5   FDHr      for_c + nad_c <=> co2_c + nadh_c
29%    8.1   DM_co2_c  --> co2_c
27%    7.63  AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
4%     1.04  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
59%   16.8   CO2tpp    co2_p <=> co2_c
28%    7.81  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
6%     1.63  HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%     1.43  CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%     1.06  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219565
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
39%   11.5   FDHr      for_c + nad_c <=> co2_c + nadh_c
31%    9     DM_co2_c  --> co2_c
26%    7.63  AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
4%     1.04  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
61%   17.7   CO2tpp    co2_p <=> co2_c
27%    7.81  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
6%     1.63  HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%     1.43  CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%     1.06  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.15474544176219565
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
38%   11.5   FDHr      for_c + nad_c <=> co2_c + nadh_c
33%    9.9   DM_co2_c  --> co2_c
25%    7.63  AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
3%     1.04  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
62%   18.6   CO2tpp    co2_p <=> co2_c
26%    7.81  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
5%     1.63  HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%     1.43  CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%     1.06  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c


### AKGDH as a reversible reaction:

In [ ]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_co2_c')
reaction.name = 'Demand carbon dioxide'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [59]:
for r in np.arange(0, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.AKGDH.bounds = (-1000, 1000)
        model.reactions.OOR3r.bounds = (0, 0)
        model.add_reaction(reaction)
        model.reactions.get_by_id('DM_co2_c').reaction = '--> co2_c'
        model.reactions.DM_co2_c.bounds = (r, 1000)
        solution = cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(solution['Biomass_Mextorquens_AM1_core'])
        #model.metabolites.co2_c.summary()

0.12260378253188665
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528


### AKGDH blocked and OOR3r added:

In [53]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_co2_c')
reaction.name = 'Demand carbon dioxide'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [54]:
for r in np.arange(0, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 0)
        model.reactions.AKGDH.bounds = (0, 0)
        model.add_reaction(reaction)
        model.reactions.get_by_id('DM_co2_c').reaction = '--> co2_c'
        model.reactions.DM_co2_c.bounds = (r, 1000)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(model.reactions.Biomass_Mextorquens_AM1_core.flux)
        model.metabolites.co2_c.summary()

0.12318677860855141
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
94%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
6%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
76%   9.95   CO2tpp    co2_p <=> co2_c
10%   1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
9%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
6%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
1%    0.142  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
0.12318677860854885
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  -------------------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
87%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
7%    1      DM_co2_c  --> co2_c
6%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
78%  11      CO2tpp    co2_p <=> co2_c
9%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
8%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
6%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
1%    0.142  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  ------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
81%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
13%   2      DM_co2_c  --> co2_c
5%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
80%  12      CO2tpp    co2_p <=> co2_c
9%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
8%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
6%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
1%    0.142  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  ------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
76%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
19%   3      DM_co2_c  --> co2_c
5%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
81%  13      CO2tpp    co2_p <=> co2_c
8%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
7%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
1%    0.142  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  ------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
72%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
23%   4      DM_co2_c  --> co2_c
5%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
82%  14      CO2tpp    co2_p <=> co2_c
8%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
7%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
1%    0.142  OOR3r     akg_c + coa_c + fdxox_c + h_c <=> co2_c + fdxrd...
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  ------------------------------

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
67%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
28%   5.1    DM_co2_c  --> co2_c
5%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
83%  15.1    CO2tpp    co2_p <=> co2_c
7%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
5%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
67%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
63%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
32%   6.2    DM_co2_c  --> co2_c
4%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
84%  16.2    CO2tpp    co2_p <=> co2_c
7%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
63%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
60%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
36%   7.3    DM_co2_c  --> co2_c
4%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
85%  17.3    CO2tpp    co2_p <=> co2_c
6%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
6%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
60%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
57%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
39%   8.4    DM_co2_c  --> co2_c
4%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
86%  18.4    CO2tpp    co2_p <=> co2_c
6%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
57%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh

PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
54%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh_c
42%   9.5    DM_co2_c  --> co2_c
4%    0.826  PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
86%  19.5    CO2tpp    co2_p <=> co2_c
6%    1.3    HCO3E     co2_c + h2o_c <=> h_c + hco3_c
5%    1.14   CCRC      b2coa_c + co2_c + nadph_c <=> 2Sethmalcoa_c + n...
4%    0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.12318677860855412
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
54%  12.2    FDHr      for_c + nad_c <=> co2_c + nadh

### Increase flux through EMC pathway and see what happens:

CCRC

In [151]:
for r in np.arange(0.723, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 0)
        model.reactions.CCRC.bounds = (r, 1000)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(r, model.reactions.Biomass_Mextorquens_AM1_core.flux)
        #model.metabolites.co2_c.summary()

0.723 0.12260378253188464
0.823 0.1226037825318869
0.9229999999999999 0.12260378184691885
1.023 0.12260378184691884
1.1229999999999998 0.12260378253188209
1.2229999999999999 0.12260378253188274
1.323 0.12260378253188306
1.4229999999999998 0.12260378253188348
1.5229999999999997 0.12260378253188457
1.6229999999999998 0.12260378253188767
1.7229999999999999 0.11748450461500881
1.8229999999999995 0.1093120298409236
1.9229999999999996 0.10113955506683853
2.0229999999999997 0.09296708029275293
2.1229999999999998 0.08479460551866784
2.2229999999999994 0.07662213074458199
2.3229999999999995 0.06844965597049689
2.4229999999999996 0.06027718119641183
2.5229999999999997 0.05210470642232648
2.6229999999999993 0.043932231648240845
2.7229999999999994 0.0357597568741552
2.8229999999999995 0.027587282100071612
2.922999999999999 0.019414807325984797
3.0229999999999992 0.01124233255189945
3.1229999999999993 0.003069857777813811


Infeasible: None (infeasible)

PPC:

In [154]:
for r in np.arange(0.998, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 0)
        model.reactions.PPC.bounds = (r, 1000)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(r, model.reactions.Biomass_Mextorquens_AM1_core.flux)
        #model.metabolites.co2_c.summary()

0.998 0.12260378071185124
1.098 0.12260378298201795
1.1980000000000002 0.1226037818457991
1.2980000000000003 0.1226037818457989
1.3980000000000004 0.12260378298201749
1.4980000000000004 0.12260378184579936
1.5980000000000005 0.12260378298201739
1.6980000000000006 0.12260378184466432
1.7980000000000007 0.12260378184579948
1.8980000000000008 0.12260378298315233
1.9980000000000009 0.12260378184579915
2.0980000000000008 0.12260378298315258
2.1980000000000013 0.12260378184466485
2.298000000000001 0.1226037818457989
2.3980000000000015 0.12260378298428767
2.498000000000001 0.12260378298201792
2.5980000000000016 0.12260378184579893
2.6980000000000013 0.12260378184579898
2.798000000000002 0.12260378184466436
2.8980000000000015 0.12260378298315261
2.998000000000002 0.12260378298315253
3.0980000000000016 0.12260378184579938
3.198000000000002 0.12260378184579893
3.298000000000002 0.12260378184579941
3.3980000000000024 0.12260378298315243
3.498000000000002 0.12260378298201777
3.5980000000000025 0.1

Infeasible: None (infeasible)

AKGDH:

In [158]:
for r in np.arange(0, 100, 1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 0)
        model.reactions.AKGDH.bounds = (-r, 0)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print('-', r, model.reactions.Biomass_Mextorquens_AM1_core.flux)
        #model.metabolites.co2_c.summary()

- 0 0.12260378253188432
- 1 0.12260378253188178
- 2 0.12260378253188178
- 3 0.12260378253188178
- 4 0.12260378253188178
- 5 0.12260378253188178
- 6 0.12260378253188178
- 7 0.12260378253188178
- 8 0.12260378253188178
- 9 0.12260378253188178
- 10 0.12260378253188178
- 11 0.12260378253188178
- 12 0.12260378253188178
- 13 0.12260378253188178
- 14 0.12260378253188178
- 15 0.12260378253188178
- 16 0.12260378253188178
- 17 0.12260378253188178
- 18 0.12260378253188178
- 19 0.12260378253188178
- 20 0.12260378253188178
- 21 0.12260378253188178
- 22 0.12260378253188178
- 23 0.12260378253188178
- 24 0.12260378253188178
- 25 0.12260378253188178
- 26 0.12260378253188178
- 27 0.12260378253188178
- 28 0.12260378253188178
- 29 0.12260378253188178
- 30 0.12260378253188178
- 31 0.12260378253188178
- 32 0.12260378253188178
- 33 0.12260378253188178
- 34 0.12260378253188178
- 35 0.12260378253188178
- 36 0.12260378253188178
- 37 0.12260378253188178
- 38 0.12260378253188178
- 39 0.12260378253188178
- 40 0.122

#### Reduced costs:

In [178]:
iMsOB3b.reactions.FALDA.bounds = (0, 0)
solution = cobra.flux_analysis.pfba(iMsOB3b)
print(iMsOB3b.reactions.Biomass_Mextorquens_AM1_core.flux)

0.12260378253188575


In [180]:
iMsOB3b.reactions.FALDA.bounds = (0, 0)
solution = model.optimize(iMsOB3b)
print(iMsOB3b.reactions.Biomass_Mextorquens_AM1_core.flux)

0.12260378253188571


In [182]:
iMsOB3b.summary()

IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  26.4   Biomass_Mext...  0.123
ch4_e      14.9       co2_e   9.97
h_e         3.26
no3_e       1.2
pi_e        0.0857
k_e         0.0226
so4_e       0.00774
mg2_e       0.00101
fe2_e       0.000984
cl_e        0.000607
ca2_e       0.000607
na1_e       0.000499
mobd_e      0.000432
cu2_e       0.000405
mn2_e       0.000405
zn2_e       0.000405
cobalt2_e   2.69e-05


In [181]:
solution.reduced_costs.to_csv(memote_directory + 'gem_confirmation/reduced_costs/' +
                    'reduced_costs_redox_arm_falda_knockout.csv')

/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:2 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [192]:
iMsOB3b.reactions.ATPS4rpp

Reaction identifier,ATPS4rpp
Name,ATP synthase
Memory address,0x07fe651ab5358
Stoichiometry,adp_c + 4.0 h_p + pi_c --> atp_c + h2o_c + 3.0 h_c ADP + 4.0 H_ + phosphate --> ATP + H2O + 3.0 H_
GPR,CQW49_19220 and CQW49_00300 and CQW49_19240 and CQW49_19235 and CQW49_00305 and CQW49_19230 and C...
Lower bound,0.0
Upper bound,1000.0


# CO2 supplementation with flux via FALDA allowed:

## With 1/3 of acetyl-CoA coming from pyruvate:

In [116]:
iMsOB3b = cobra.io.load_json_model(memote_directory + draft_gems_dir_8 + "bigg_OB3b_12_02_2020.json")

In [100]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_co2_c')
reaction.name = 'Demand carbon dioxide'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [103]:
for r in np.arange(0, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 1000)
        model.add_reaction(reaction)
        model.reactions.get_by_id('DM_co2_c').reaction = '--> co2_c'
        model.reactions.DM_co2_c.bounds = (r, 1000)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(model.reactions.Biomass_Mextorquens_AM1_core.flux)
        #model.metabolites.co2_c.summary()

0.12260378253188592
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531
0.12260378253188531


## With reverse TCA:

### With OOR3r added:

In [139]:
iMsOB3b = cobra.io.load_json_model(memote_directory + draft_gems_dir_8 + "bigg_OB3b_12_02_2020_TCA_rev.json")

In [105]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_co2_c')
reaction.name = 'Demand carbon dioxide'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [107]:
for r in np.arange(0, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 1000)
        model.add_reaction(reaction)
        model.reactions.get_by_id('DM_co2_c').reaction = '--> co2_c'
        model.reactions.DM_co2_c.bounds = (r, 1000)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(model.reactions.Biomass_Mextorquens_AM1_core.flux)
        #model.metabolites.co2_c.summary()

0.1646933968484332
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.16469339684844
0.1646933968

### AKGDH as a reversible reaction:

In [108]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_co2_c')
reaction.name = 'Demand carbon dioxide'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [109]:
for r in np.arange(0, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 1000)
        model.reactions.AKGDH.bounds = (-1000, 1000)
        model.reactions.OOR3r.bounds = (0, 0)
        model.add_reaction(reaction)
        model.reactions.get_by_id('DM_co2_c').reaction = '--> co2_c'
        model.reactions.DM_co2_c.bounds = (r, 1000)
        solution = cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(solution['Biomass_Mextorquens_AM1_core'])
        #model.metabolites.co2_c.summary()

0.12260378252801624
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528
0.12260378253188528


### AKGDH blocked and OOR3r added:

In [110]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_co2_c')
reaction.name = 'Demand carbon dioxide'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [112]:
for r in np.arange(0, 10, 0.1):
    with iMsOB3b as model:
        model.reactions.FALDA.bounds = (0, 1000)
        model.reactions.AKGDH.bounds = (0, 0)
        model.add_reaction(reaction)
        model.reactions.get_by_id('DM_co2_c').reaction = '--> co2_c'
        model.reactions.DM_co2_c.bounds = (r, 1000)
        cobra.flux_analysis.pfba(model).fluxes
        #model.summary()
        print(model.reactions.Biomass_Mextorquens_AM1_core.flux)
        #model.metabolites.co2_c.summary()

0.12318677860855343
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858
0.12318677860853858


# Check the fate of succinyl-coA/Reverse TCA:

## Check what happens with succinyl-coA under regular FBA analysis:

In [124]:
for reaction in iMsOB3b.reactions:
    reaction.annotation = {}

In [87]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 0)
    iMsOB3b.reactions.AKGDH.bounds = (-1000, 1000)
    cobra.flux_analysis.pfba(model).fluxes
    model.summary()
    model.metabolites.succoa_c.summary()

IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  26.4   Biomass_Mext...  0.123
ch4_e      14.9       co2_e   9.97
h_e         3.09
no3_e       1.2
pi_e        0.0857
k_e         0.0226
so4_e       0.00774
mg2_e       0.00101
fe2_e       0.000984
ca2_e       0.000607
cl_e        0.000607
na1_e       0.000499
mobd_e      0.000432
cu2_e       0.000405
mn2_e       0.000405
zn2_e       0.000405
cobalt2_e   2.69e-05
PRODUCING REACTIONS -- succinyl_CoA (succoa_c)
----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%  1.13    MMM2r     mmcoa_R_c <=> succoa_c

CONSUMING REACTIONS -- succinyl_CoA (succoa_c)
----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
85%   0.961   OCOAT1r   acac_c + succoa_c <=> aa

In [11]:
iMsOB3b.metabolites.acac_c.name

'acetoacetate'

### With FALDA on:

In [117]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 1000)
    iMsOB3b.reactions.AKGDH.bounds = (-1000, 1000)
    cobra.flux_analysis.pfba(model).fluxes
    model.summary()
    model.metabolites.succoa_c.summary()

IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  26.4   Biomass_Mext...  0.123
ch4_e      14.9       co2_e   9.97
h_e         3.09
no3_e       1.2
pi_e        0.0857
k_e         0.0226
so4_e       0.00774
mg2_e       0.00101
fe2_e       0.000984
ca2_e       0.000607
cl_e        0.000607
na1_e       0.000499
mobd_e      0.000432
cu2_e       0.000405
mn2_e       0.000405
zn2_e       0.000405
cobalt2_e   2.69e-05
PRODUCING REACTIONS -- succinyl_CoA (succoa_c)
----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%  1.13    MMM2r     mmcoa_R_c <=> succoa_c

CONSUMING REACTIONS -- succinyl_CoA (succoa_c)
----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
85%   0.961   OCOAT1r   acac_c + succoa_c <=> aa

Check succinyl CoA 3 oxo acid CoA transferase enzyme:

In [285]:
iMsOB3b.reactions.OCOAT1r

Reaction identifier,OCOAT1r
Name,succinyl CoA 3 oxo acid CoA transferase
Memory address,0x07f2ec3a17c88
Stoichiometry,acac_c + succoa_c <=> aacoa_c + succ_c acetoacetate + succinyl_CoA <=> acetoacetyl_CoA + succinate
GPR,CQW49_18630
Lower bound,-1000.0
Upper bound,1000.0


## Reverse TCA, AKGDH and OOR3r included:

In [129]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 0)
    model.reactions.AKGDH.bounds = (-1000, 1000)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(memote_directory + 'gem_confirmation/additional_constraints/' +
                    'reverse_TCA.csv')
    model.summary()
    model.metabolites.oaa_c.summary()

/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  29.5   Biomass_Mext...  0.155
ch4_e      14.9       co2_e   8.68
no3_e       1.51
pi_e        0.108
k_e         0.0285
so4_e       0.00977
mg2_e       0.00128
fe2_e       0.00124
ca2_e       0.000766
cl_e        0.000766
na1_e       0.00063
mobd_e      0.000545
cu2_e       0.000511
mn2_e       0.000511
zn2_e       0.000511
cobalt2_e   3.4e-05
PRODUCING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
100%   1.06   PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c

CONSUMING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
67%    0.711  MDH       mal_L_c + nad_c <=> h_c + nadh_c + oaa_c
33%  

### With FALDA on:

In [130]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 1000)
    model.reactions.AKGDH.bounds = (-1000, 1000)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(memote_directory + 'gem_confirmation/additional_constraints/' +
                    'reverse_TCA_FALDA_on.csv')
    model.summary()
    model.metabolites.oaa_c.summary()

IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  30.4   Biomass_Mext...  0.165
ch4_e      14.9       co2_e   8.28
no3_e       1.61
pi_e        0.115
k_e         0.0304
so4_e       0.0104
mg2_e       0.00136
fe2_e       0.00132
cl_e        0.000815
ca2_e       0.000815
na1_e       0.00067
mobd_e      0.00058
cu2_e       0.000543
mn2_e       0.000543
zn2_e       0.000543
cobalt2_e   3.62e-05
PRODUCING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
100%   1.12   PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c

CONSUMING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
67%    0.756  MDH       mal_L_c + nad_c <=> h_c + nadh_c + oaa_c
33%   

/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


## AKGDH blocked and OOR3r included:

In [135]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 0)
    model.reactions.AKGDH.bounds = (0, 0)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(memote_directory + 'gem_confirmation/additional_constraints/' +
                    'reverse_TCA_AKGDH_blocked_OOR3r_included.csv')
    model.metabolites.oaa_c.summary()

PRODUCING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
100%   0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c

CONSUMING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
67%    0.566  MDH       mal_L_c + nad_c <=> h_c + nadh_c + oaa_c
33%    0.276  ASPTA     akg_c + asp_L_c <=> glu_L_c + oaa_c


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


### With FALDA on:

In [136]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 1000)
    model.reactions.AKGDH.bounds = (0, 0)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(memote_directory + 'gem_confirmation/additional_constraints/' +
                    'reverse_TCA_AKGDH_blocked_OOR3r_included_FALDA_on.csv')
    model.metabolites.oaa_c.summary()

PRODUCING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
100%   0.841  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c

CONSUMING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
67%    0.566  MDH       mal_L_c + nad_c <=> h_c + nadh_c + oaa_c
33%    0.276  ASPTA     akg_c + asp_L_c <=> glu_L_c + oaa_c


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


## AKGDH is reversible and OOR3r blocked:

In [144]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 0)
    model.reactions.OOR3r.bounds = (0, 0)
    model.reactions.AKGDH.bounds = (-1000, 1000)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(memote_directory + 'gem_confirmation/additional_constraints/' +
                    'reverse_TCA_AKGDH_reversible_OOR3r_blocked.csv')
    model.metabolites.oaa_c.summary()

PRODUCING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
100%   0.837  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c

CONSUMING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------
67%    0.563  MDH       mal_L_c + nad_c <=> h_c + nadh_c + oaa_c
33%    0.274  ASPTA     akg_c + asp_L_c <=> glu_L_c + oaa_c


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


### With FALDA on:

In [145]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 1000)
    model.reactions.OOR3r.bounds = (0, 0)
    model.reactions.AKGDH.bounds = (0, 0)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(memote_directory + 'gem_confirmation/additional_constraints/' +
                    'reverse_TCA_AKGDH_reversible_OOR3r_blocked_FALDA_on.csv')
    model.metabolites.oaa_c.summary()

PRODUCING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
100%   0.943  PPC       co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c

CONSUMING REACTIONS -- oxaloacetate (oaa_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
56%    0.528  MDH       mal_L_c + nad_c <=> h_c + nadh_c + oaa_c
29%    0.274  ASPTA     akg_c + asp_L_c <=> glu_L_c + oaa_c
15%    0.141  CS        accoa_c + h2o_c + oaa_c <=> cit_c + coa_c + h_c


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


## Check whether significant amount of CO2 released is incorporated via EMC pathway:

In [137]:
with iMsOB3b as model:
    model.reactions.FALDA.bounds = (0, 0)
    model.reactions.HCO3E.bounds = (0, 1000)
    cobra.flux_analysis.pfba(model).fluxes
    model.summary()
    model.metabolites.co2_c.summary()

IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  29.5   Biomass_Mext...  0.155
ch4_e      14.9       co2_e   8.68
no3_e       1.51
pi_e        0.108
k_e         0.0285
so4_e       0.00977
mg2_e       0.00128
fe2_e       0.00124
ca2_e       0.000766
cl_e        0.000766
na1_e       0.00063
mobd_e      0.000545
mn2_e       0.000511
cu2_e       0.000511
zn2_e       0.000511
cobalt2_e   3.4e-05
PRODUCING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  --------------------------------------------------
57%  11.5    FDHr      for_c + nad_c <=> co2_c + nadh_c
38%   7.63   AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
5%    1.04   PDH       coa_c + nad_c + pyr_c --> accoab_c + co2_c + na...
1%    0.126  ACLS      h_c + 2.0 pyr_c <=> alac_S_c + co2_c

CONSUMING REACTIONS -- CO2 (co2_c)
----------------------------------
%      FL

In [72]:
iMsOB3b.reactions.PPCOAC

Reaction identifier,PPCOAC
Name,propionyl CoA carboxylase 1
Memory address,0x07fe65c5d6240
Stoichiometry,atp_c + hco3_c + ppcoa_c --> adp_c + h_c + mmcoa_S_c + pi_c ATP + HCO3_ + propionyl_CoA --> ADP + H_ + S__methylmalonyl_CoA + phosphate
GPR,CQW49_20805 and CQW49_16085
Lower bound,0.0
Upper bound,1000.0
